In [3]:
import torch
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import random
import text_util
import openai
from openai_llm import oai_get_response

from openai import OpenAI
from agatha.construct.semrep_handler import SemRepHandler
from agatha.util.sqlite3_lookup import Sqlite3LookupTable
from sentence_transformers import SentenceTransformer, util

from utils import retrieve_abstracts_from_pmids, select_initial_pmids, get_pmids_with_abstracts, update_pmids, find_top_documents_for_queries
from agatha_evaluator import AgathaEvaluator
from feedback_loop import FeedbackLoop
from openai_llm import oai_get_response

import warnings
warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
agatha_sent_db_path = (
    '/lustre/acslab/shared/Agatha_shared/2021_11_22_full/sentences.sqlite3'
)
sents_db = Sqlite3LookupTable(agatha_sent_db_path)

# Define paths
workingFolder = '/lustre/acslab/shared/Agatha_shared/2021_11_22'
model_path = f'{workingFolder}/model_epoch5_darwin.pt'
embedding_path = f'{workingFolder}/embeddings/predicate_subset'
entity_db = f'{workingFolder}/predicate_entities.sqlite3'
graph_db = f'{workingFolder}/predicate_graph.sqlite3'

# Initialize the evaluator to load the model and predicate CUI pairs
evaluator = AgathaEvaluator(
    model_path=model_path,
    embedding_path=embedding_path,
    entity_db=entity_db,
    graph_db=graph_db
)

# Initialize FeedbackLoop
feedback_loop = FeedbackLoop(
    model=evaluator.model,
    pred_cui_pairs_set=evaluator.pred_cui_pairs_set,
    sents_db=sents_db,
    evaluator=evaluator
)

Existing services killed.

SKR and WSD services started.


100%|██████████| 26535915/26535915 [00:37<00:00, 709491.73it/s]


In [5]:
df = pd.read_pickle('P3_SLC6A3_Bupropion.pkl')
p3_top_20_df = df.loc[:].head(20)
p3_top_20_df

,path,score_std,score_mean,dec_path,context_pmids
220,"[C1420213, C0013124, C0424166, C0085208]",2.463338e-06,0.999911,"[SLC6A3 gene, Drinking behavior processes, Soc...","{('C1420213', 'C0013124'): ['19352220'], ('C00..."
339,"[C1420213, C1417965, C0030662, C0085208]",6.882552e-08,0.999898,"[SLC6A3 gene, OPRM1 gene, Gambling, Pathologic...","{('C1420213', 'C1417965'): ['9790747', '225510..."
114,"[C1420213, C1417965, C0001969, C0085208]",0.000000e+00,0.999884,"[SLC6A3 gene, OPRM1 gene, Alcoholic Intoxicati...","{('C1420213', 'C1417965'): ['9790747', '225510..."
195,"[C1420213, C1439340, C3539788, C0085208]",2.038944e-05,0.999809,"[SLC6A3 gene, SLC6A3 protein, human, Synaptic ...","{('C1420213', 'C1439340'): ['17171650', '32102..."
180,"[C1420213, C1439340, C0020651, C0085208]",5.052018e-05,0.999795,"[SLC6A3 gene, SLC6A3 protein, human, Hypotensi...","{('C1420213', 'C1439340'): ['17171650', '32102..."
141,"[C1420213, C0013124, C0023896, C0085208]",0.000000e+00,0.999781,"[SLC6A3 gene, Drinking behavior processes, Alc...","{('C1420213', 'C0013124'): ['19352220'], ('C00..."
102,"[C1420213, C1417965, C0221745, C0085208]",0.000000e+00,0.999772,"[SLC6A3 gene, OPRM1 gene, Depression suicidal,...","{('C1420213', 'C1417965'): ['9790747', '225510..."
309,"[C1420213, C1760428, C0085208]",0.000000e+00,0.999764,"[SLC6A3 gene, Suicidal behavior, bupropion]","{('C1420213', 'C1760428'): ['34199792'], ('C17..."
203,"[C1420213, C1417965, C0270549, C0085208]",0.000000e+00,0.999724,"[SLC6A3 gene, OPRM1 gene, Generalized Anxiety ...","{('C1420213', 'C1417965'): ['9790747', '225510..."
205,"[C1420213, C0001957, C0527316, C0085208]",0.000000e+00,0.999707,"[SLC6A3 gene, Alcohol Withdrawal Delirium, don...","{('C1420213', 'C0001957'): ['12513948'], ('C00..."


In [6]:
%%time
result_df = feedback_loop.feedback_loop_all_paths(p3_top_20_df)


-------------------------Processing Path 220-------------------------------------------

Here are all the PMIDs for path 220: {('C1420213', 'C0013124'): ['19352220'], ('C0013124', 'C0424166'): ['30138776', '29175292', '24841178', '26235528'], ('C0424166', 'C0085208'): ['28591669']}


100%|██████████| 3/3 [00:00<00:00, 24244.53it/s]


Here are the selected PMIDs for context: ['19352220', '30138776', '29175292', '24841178', '28591669']


Retrieving abstracts: 100%|██████████| 5/5 [00:00<00:00, 16082.45it/s]


------------------Step 2 Prepare prompt + context -----------------------------------

How would you describe an indirect relationship between GABBR1 and Baclofen given the following scientific abstracts as context?

Association between harmful alcohol consumption behavior and dopamine transporter (DAT1) gene polymorphisms in a male Finnish population. Ethanol-induced dopamine (DA) release in the mesolimbic system may reinforce excessive alcohol intake and the progression of alcohol dependence. Within this reward system, the DA transporter (DAT1) plays a key role in the regulation of dopaminergic neurotransmission through presynaptic DA reuptake. This study investigated whether DAT1 genetic variation was associated with either alcohol consumption behavior or alcohol dependence in a Finnish cohort. Eight single nucleotide polymorphisms and a frequently studied 3'-untranslated region 40-bp variable number tandem repeat were genotyped in unrelated male Finnish participants selected from 

...

100%|██████████| 3/3 [00:00<00:00, 14838.34it/s]


Here are the selected PMIDs for context: ['9790747', '22551036', '20668445', '33806634']


Retrieving abstracts: 100%|██████████| 4/4 [00:00<00:00, 12291.00it/s]


------------------Step 2 Prepare prompt + context -----------------------------------

How would you describe an indirect relationship between GABBR1 and Baclofen given the following scientific abstracts as context?

Population studies of polymorphisms at loci of neuropsychiatric interest (tryptophan hydroxylase (TPH), dopamine transporter protein (SLC6A3), D3 dopamine receptor (DRD3), apolipoprotein E (APOE), mu opioid receptor (OPRM1), and ciliary neurotrophic factor (CNTF)). We determined allele frequencies for polymorphisms at several loci of interest in neuropsychiatry-tryptophan hydroxylase (TPH), dopamine transporter protein (SLC6A3), D3 dopamine receptor (DRD3), apolipoprotein E (APOE), ciliary neurotrophic factor (CNTF), and the mu opioid receptor (OPRM1)-in samples of individuals from populations in several different parts of the world. Associations with psychiatric illness have been proposed for specific polymorphisms at TPH (suicide-related behaviors and impulsivity), DRD3

Response from LLM at iteration 1:
Given the provided abstracts, an indirect relationship between GABBR1 and Baclofen can be described as follows:

1. **GABBR1 and Neuropsychiatric Disorders**: The first abstract discusses polymorphisms at various loci of neuropsychiatric interest, including genes like OPRM1, SLC6A3, and others. While GABBR1 is not directly mentioned, it is known to be involved in neuropsychiatric conditions through its role in the GABAergic system. Baclofen, a GABA_B receptor agonist, would interact with the GABA_B receptor, which is composed of GABBR1 and GABBR2 subunits.

2. **OPRM1 and Substance Dependence**: The second and third abstracts focus on the OPRM1 gene and its role in substance dependence and response to treatments like naltrexone and methadone. Although these studies do not directly mention GABBR1, they highlight the importance of genetic variations in treatment responses for neuropsychiatric conditions. Baclofen, used for treating alcohol dependence and